# Using Jurassic-2 Jumbo Instruct on SageMaker through Model Packages

This sample notebook shows you how to deploy **Jurassic-2 Jumbo Instruct** using Amazon SageMaker.

> **Note**: This is a reference notebook and it cannot run unless you make changes suggested in the notebook.

## Pre-requisites:
1. Before running this notebook, please make sure you got this notebook from the model catalog on SageMaker AWS Management Console.
1. **Note**: This notebook contains elements which render correctly in Jupyter interface. Open this notebook from an Amazon SageMaker Notebook Instance or Amazon SageMaker Studio.
1. Ensure that IAM role used has **AmazonSageMakerFullAccess**.
1. This notebook is intended to work with **boto3 v1.25.4** or higher.

## Contents:
1. [Select model package](#1.-Select-model-package)
2. [Create an endpoint and perform real-time inference](#2.-Create-an-endpoint-and-perform-real-time-inference)
   1. [Create an endpoint](#A.-Create-an-endpoint)
   2. [Interact with the model](#B.-Interact-with-the-model)
   3. [Give it an instruction](#C.-Give-it-an-instruction)
   4. [Delete the endpoint](#D.-Delete-the-endpoint)
3. [Clean-up](#4.-Clean-up)
    1. [Delete the model](#A.-Delete-the-model)
    

## Usage instructions
You can run this notebook one cell at a time (By using Shift+Enter for running a cell).

## 1. Select model package
Confirm that you received this notebook from the model catalog in SageMaker AWS Management Console.

In [1]:
model_package_map = {
    "us-east-1": "arn:aws:sagemaker:us-east-1:416407187090:model-package/j2-jumbo-instruct-v1-0-20"
}

In [2]:
import json
from sagemaker import ModelPackage
from sagemaker import get_execution_role
from sagemaker import ModelPackage
import sagemaker as sage
import boto3

### Check the version of boto3 - must be v1.25.4 or higher
If you see a lower version number, pick another kernel to run the notebook, with Python 3.8 or above

In [3]:
boto3.__version__

'1.26.74'

### Install ai21 python SDK

In [4]:
! pip install -U "ai21[SM]"
import ai21

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com


In [5]:
region = boto3.Session().region_name
if region not in model_package_map.keys():
    raise ("UNSUPPORTED REGION")

model_package_arn = model_package_map[region]

In [6]:
role = get_execution_role()
sagemaker_session = sage.Session()

runtime_sm_client = boto3.client("runtime.sagemaker")

## 2. Create an endpoint and perform real-time inference

If you want to understand how real-time inference with Amazon SageMaker works, see [Documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/deploy-model.html).

In [7]:
model_name = "j2-jumbo-instruct"

content_type = "application/json"

real_time_inference_instance_type = (
    "ml.g5.48xlarge"
)

### A. Create an endpoint

In [8]:
# create a deployable model from the model package.
model = ModelPackage(
    role=role, model_package_arn=model_package_arn, sagemaker_session=sagemaker_session
)

# Deploy the model
predictor = model.deploy(1, real_time_inference_instance_type, endpoint_name=model_name, 
                         model_data_download_timeout=3600,
                         container_startup_health_check_timeout=600,
                        )

----------------------------------------!

Once endpoint has been created, you would be able to perform real-time inference.

### B. Interact with the model

You can think of Jurassic-2 Jumbo Instruct as a smart auto-completion algorithm: give it some text as input and it will generate some relevant text to naturally complete your input.

These two helpful concepts are worth being familiar with:
- **Prompt** - the input you provide to the model.
- **Completion** - the output text the model generates.

Enter a simple prompt: "To be, or", and let the model complete it

In [9]:
response = ai21.Completion.execute(sm_endpoint="j2-jumbo-instruct",
                                   prompt="To be, or",
                                   maxTokens=4,
                                   temperature=0,
                                   numResults=1)

print(response['completions'][0]['data']['text'])

 not to be, that is the question


As you can see, the model identifies the beginning of a famous quote, and completes it correctly.

### C. Give it an instruction

This model was specifically trained to follow natural language instructions. It is the most natural way to interact with large language models: simply tell the model what you want it to do, and it will follow.

For this notebook, we will apply the model to summarize technology articles. Simply tell the model what you want it to do with an instruction in plain English.

In [12]:
instruction = """Below is some text:
Meet Carbon Maps, a new French startup that raised $4.3 million (€4 million) just a few weeks after its inception. The company is building a software-as-a-service platform for the food industry so that they can track the environmental impact of each of their products in their lineup. The platform can be used as a basis for eco ratings.

While there are quite a few carbon accounting startups like Greenly, Sweep, Persefoni and Watershed, Carbon Maps isn’t an exact competitor as it doesn’t calculate a company’s carbon emissions as a whole. It doesn’t focus on carbon emissions exclusively either. Carbon Maps focuses on the food industry and evaluates the environmental impact of products — not companies.

Co-founded by Patrick Asdaghi, Jérémie Wainstain and Estelle Huynh, the company managed to raise a seed round with Breega and Samaipata — these two VC firms already invested in Asdaghi’s previous startup, FoodChéri.

FoodChéri is a full-stack food delivery company that designs its own meals and sells them directly to end customers with an important focus on healthy food. It also operates Seazon, a sister company for batch deliveries. The startup was acquired by Sodexo a few years ago.

“On the day that I left, I started working on food and health projects again,” Asdaghi told me. “I wanted to make an impact, so I started moving up the supply chain and looking at agriculture.”

And the good news is that Asdaghi isn’t the only one looking at the supply chain of the food industry. In France, some companies started working on an eco-score with a public agency (ADEME) overseeing the project. It’s a life cycle assessment that leads to a letter rating from A to E.

While very few brands put these letters on their labels, chances are companies that have good ratings will use the eco-score as a selling point in the coming years.


But these ratings could become even more widespread as regulation is still evolving. The European Union is even working on a standard — the Product Environmental Footprint (PEF). European countries can then create their own scoring systems based on these European criteria, meaning that food companies will need good data on their supply chains.

“The key element in the new eco-score that’s coming up is that there will be some differences within a product category because ingredients and farming methods are different,” Asdaghi said. “It’s going to take into consideration the carbon impact, but also biodiversity, water consumption and animal welfare.”

For instance, when you look at ground beef, it’s extremely important to know whether farmers are using soy from Brazil or grass to feed cattle.

“We don’t want to create the ratings. We want to create the tools that help with calculations — a sort of SAP,” Asdaghi said.

So far, Carbon Maps is working with two companies on pilot programs as it’s going to require a ton of work to cover each vertical in the food industry. The startup creates models with as many criteria as possible to calculate the impact of each criteria. It uses data from standardized sources like GHG Protocol, IPCC, ISO 14040 and 14044.

The company targets food brands because they design the recipes and select their suppliers. Eventually, Carbon Maps hopes that everybody across the supply chain is going to use its platform in one way or another.

“You can’t have a true climate strategy if you don’t have some collaboration across the chain,” Asdaghi said.

##

Summarize the above text in a few bullets.

"""

response = ai21.Completion.execute(sm_endpoint="j2-jumbo-instruct",
                                   prompt=instruction,
                                   maxTokens=100,
                                   temperature=0.2,
                                   numResults=1)

print(response['completions'][0]['data']['text'])

- Carbon Maps is a startup that raised $4.3 million (€4 million) just a few weeks after its inception.
- The company is building a software-as-a-service platform for the food industry so that they can track the environmental impact of each of their products in their lineup.
- The platform can be used as a basis for eco ratings.


#### Be specific in your prompt
The model was asked to summarize the article in several bullet points. Depending on your requirements, you may require a specific number of bullets. You can ask the model to stick to it (in this case, 5 bullets):

In [13]:
instruction = """Below is some text:
Meet Carbon Maps, a new French startup that raised $4.3 million (€4 million) just a few weeks after its inception. The company is building a software-as-a-service platform for the food industry so that they can track the environmental impact of each of their products in their lineup. The platform can be used as a basis for eco ratings.

While there are quite a few carbon accounting startups like Greenly, Sweep, Persefoni and Watershed, Carbon Maps isn’t an exact competitor as it doesn’t calculate a company’s carbon emissions as a whole. It doesn’t focus on carbon emissions exclusively either. Carbon Maps focuses on the food industry and evaluates the environmental impact of products — not companies.

Co-founded by Patrick Asdaghi, Jérémie Wainstain and Estelle Huynh, the company managed to raise a seed round with Breega and Samaipata — these two VC firms already invested in Asdaghi’s previous startup, FoodChéri.

FoodChéri is a full-stack food delivery company that designs its own meals and sells them directly to end customers with an important focus on healthy food. It also operates Seazon, a sister company for batch deliveries. The startup was acquired by Sodexo a few years ago.

“On the day that I left, I started working on food and health projects again,” Asdaghi told me. “I wanted to make an impact, so I started moving up the supply chain and looking at agriculture.”

And the good news is that Asdaghi isn’t the only one looking at the supply chain of the food industry. In France, some companies started working on an eco-score with a public agency (ADEME) overseeing the project. It’s a life cycle assessment that leads to a letter rating from A to E.

While very few brands put these letters on their labels, chances are companies that have good ratings will use the eco-score as a selling point in the coming years.


But these ratings could become even more widespread as regulation is still evolving. The European Union is even working on a standard — the Product Environmental Footprint (PEF). European countries can then create their own scoring systems based on these European criteria, meaning that food companies will need good data on their supply chains.

“The key element in the new eco-score that’s coming up is that there will be some differences within a product category because ingredients and farming methods are different,” Asdaghi said. “It’s going to take into consideration the carbon impact, but also biodiversity, water consumption and animal welfare.”

For instance, when you look at ground beef, it’s extremely important to know whether farmers are using soy from Brazil or grass to feed cattle.

“We don’t want to create the ratings. We want to create the tools that help with calculations — a sort of SAP,” Asdaghi said.

So far, Carbon Maps is working with two companies on pilot programs as it’s going to require a ton of work to cover each vertical in the food industry. The startup creates models with as many criteria as possible to calculate the impact of each criteria. It uses data from standardized sources like GHG Protocol, IPCC, ISO 14040 and 14044.

The company targets food brands because they design the recipes and select their suppliers. Eventually, Carbon Maps hopes that everybody across the supply chain is going to use its platform in one way or another.

“You can’t have a true climate strategy if you don’t have some collaboration across the chain,” Asdaghi said.

##

Summarize the above text in 5 bullets.

"""

response = ai21.Completion.execute(sm_endpoint="j2-jumbo-instruct",
                                   prompt=instruction,
                                   maxTokens=100,
                                   temperature=0.2,
                                   numResults=1)

print(response['completions'][0]['data']['text'])

- Carbon Maps is a startup that raised $4.3 million.
- The company builds a software-as-a-service platform for the food industry.
- The platform can be used as a basis for eco ratings.
- Carbon Maps focuses on the food industry and evaluates the environmental impact of products.
- The startup targets food brands because they design the recipes and select their suppliers.


#### More format restrictions?
If you have several examples of your optimal output, you can feed the model with a few-shot prompt that includes several examples. Prompts like this are very useful when you want to enforce a certain format, usually a specific and unique one. These prompts work best with base models; try to use them with [Jurassic-2 Jumbo](https://github.com/AI21Labs/SageMaker/blob/main/J2_Jumbo_example_model_use.ipynb) instead.

### Interested in learning more?
Take a look at our [blog post](https://www.ai21.com/blog/building-cv-profile-generator-using-ai21-studio) to understand the process of building a good prompt.

### D. Delete the endpoint

Now that you have successfully performed a real-time inference, you do not need the endpoint any more. You can terminate the endpoint to avoid being charged.

In [22]:
model.sagemaker_session.delete_endpoint(model_name)
model.sagemaker_session.delete_endpoint_config(model_name)

## 4. Clean-up

### A. Delete the model

In [23]:
model.delete_model()